<a href="https://colab.research.google.com/github/neohack22/IASD/blob/graphs/graphs/WS_22_02_IASDCo_AuthorGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Install the necessary library in your Colab notebook environment and connect to your hosted Neo4J Sandbox.

In [ ]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 172 kB 2.1 MB/s 
  Created wheel for neo4j: filename=neo4j-5.0.0-py3-none-any.whl size=247343 sha256=9fe9b2f9e66a99aa864a6a3c3e36ae45e3bb143e2d372aac8f535934a867b6f6
  Stored in directory: /root/.cache/pip/wheels/68/a8/b6/0f1d16f8ba65752b567b654b8cf4d3901b9fe94150c47f827d
Successfully built neo4j


In [ ]:
ip = "54.174.38.179"
bolt_port = "7687"
username = "neo4j"
password = "spots-carrier-wires"

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://" + ip + ":" + bolt_port, auth=(username, password))

print(driver.address) # your-sandbox-ip:your-sandbox-bolt-port

54.174.38.179:7687


# Build the co-author graph

We are going to build an inferred graph of co-authors based on *Author*s that have collaborated on the same papers. 

We are also going to store a property on the *CO_AUTHOR* relationship to indicate the year of their first collaboration and another one to indicate the number of their collaborations.

Complete the query below to create the CO_AUTHOR relationships, each one with *year* and *collaborations* properties. Feel free to experiment in the Neo4J Browser first.

Note: we will be using the [apoc.periodic.iterate](https://neo4j.com/labs/apoc/4.1/overview/apoc.periodic/apoc.periodic.iterate/) procedure to commit in batches to the DB.


In [ ]:
query = """
CALL apoc.periodic.iterate(

  " 
  /*
   Step 1: Co-author pattern matching
  */

  MATCH ... // Use "a1" and "a2" naming to handle the matched Authors nodes
  WITH a1, a2, paper 
  ORDER BY ... // Ensure we consider the oldest collab between authors to get the year of their first collaboration 
  RETURN a1, a2, ... AS year, ... AS nb_collaborations 
  ",

  "
  /* 
  Step 2: Create CO_AUTHOR relationships and set properties
  */
  
  MERGE ... // Use a "coauthor" naming to handle the CO_AUTHOR relationship
  SET coauthor.collaborations = ...
  ",

  {batchSize: 100})
"""

In [ ]:
#@title Hint

query = """
CALL apoc.periodic.iterate(

  "
  MATCH (a1: ... )<-[...]-(paper)-[...]->(a2: ...)
  WITH a1, a2, paper 
  ORDER BY a1, paper.year  
  RETURN a1, a2, collect(paper)[0].year AS year, ... AS nb_collaborations
  ",

  "
  MERGE (a1)-[...]-(a2)
  SET coauthor.collaborations = ... 
  ",

  {batchSize: 100})
"""

In [ ]:
with driver.session() as session:
  result = session.run(query)
  for row in result:
    print(row)

<Record batches=3105 total=310448 timeTaken=25 committedOperations=310448 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 3105, 'committed': 3105, 'failed': 0, 'errors': {}} operations={'total': 310448, 'committed': 310448, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 0, 'relationshipsCreated': 11633, 'nodesCreated': 0, 'propertiesSet': 322081, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


# Queries

- Check the previous query results by using this new "CO_AUTHOR" relationship. 

Previous query: Find the Author with whom "Salvatore Greco" has co-authored the most with.

Hint: use the *collaboration* property.

- Recommend a new author to collaborate with Salvatore Greco based on the collaborations of his co-authors. 

Hint: find and rank Salvatore's co-authors' own co-authors, with whom Salvatore has not collaborated yet.

In [ ]:
#@title Solution

MATCH (a:Author {name:'Salvatore Greco'})-[r1:CO_AUTHOR]->(ca:Author)-[r2:CO_AUTHOR]->(caofca:Author)
WITH a, caofca, r1.collaborations as nb_ca_collabs, r2.collaborations as nb_caofca_collabs
WHERE a <> caofca AND NOT (a)-[:CO_AUTHOR]-(caofca)
RETURN caofca.name, nb_ca_collabs, nb_caofca_collabs 
ORDER BY nb_ca_collabs DESC, nb_caofca_collabs DESC 
LIMIT 1

Now that we have created our co-author graph, we want to come up with an approach that will allow us to predict future links (relationships) that will be created between people.